In [1]:
import torch 
import numpy as np

from models import Unet

from preprocessing import (
    subsample_point_cloud, 
    create_digital_elevation_model,
    pad_multiplo,
    create_input)

from postprocessing import(
    get_binary_img,
    get_componets,
    get_segments,
    get_measurement,
    get_coords,
    check_pretil
)

class PretilNet:
    def __init__(self, weights="model.pth", device='cuda'):
        self.model = Unet()
        self.model.load_state_dict(torch.load(weights))
        self.device = device
        self.model.eval()
        if device == 'cuda':
            self.model.cuda()


    def __call__(self, las_file, kmz_pretil, factor_size=0.1):
        ## Preprocessing
        dem_info = self.__preprocessing(las_file, factor_size)
        input = create_input(dem_info['dem']).astype(np.float32)
        input = torch.tensor(input, dtype=torch.float32, device=self.device)

        ## IA
        outputs = self.model(input)  # Returns a list of tensors.

        ## Posprocessing
        data_pixel = self.__postprocessing(outputs, dem_info['mask'])
        data_coords = get_coords(data_pixel, dem_info)
        results = check_pretil(kmz_pretil, data_coords)
        return results
    
    def __preprocessing(self, input_file, factor_size):
        las = subsample_point_cloud(input_file, factor=factor_size)
        dem_info = create_digital_elevation_model(las.x,las.y,las.z) 
        dem_info['dem'], dem_info['mask'] = pad_multiplo(dem_info['dem'], dem_info['mask'])
        return dem_info

    def __postprocessing(self, outputs, mask):
        mask = np.expand_dims(mask, -1)
        outputs = [o.detach().cpu().numpy() for o in outputs]
        outputs = [o[0,:,:,:].transpose(1, 2, 0) * mask for o in outputs]
        
        img_class = outputs[0]
        img_pretil = outputs[1]

        binary_img = get_binary_img(img_class)
        valid_components, labels = get_componets(binary_img)
        segments = get_segments(valid_components, labels)
        binary_mask = binary_img.astype(bool)
        results = get_measurement(segments, binary_mask, img_pretil)

        return results


In [2]:
import os
import json

model = PretilNet(weights="weigths/model_epoch_79.pth")

path_kmz = 'Primer_Anillo_Aeropuerto.kmz'
paths_las = [f'../data/{i}' for i in os.listdir('../data')]

for path_las in paths_las:
    path_json = f'../results/{path_las.split("/")[-1].split(".")[0]}.json'

    outputs = model(path_las, path_kmz, factor_size=1)

    result = {}
    for key, values in outputs.items():
        result[key] = []
        for value in values:
            lat, lon = value[0]
            weight = value[1]
            height = value[2][0]
            check = height > 2 or (weight < 2.5 or weight < 3.5)
            result[key].append({
                "lat": lat,
                "lon": lon,
                "weight": weight,
                "height": height,
                "check": str(check)
            })

    # Guardar el resultado en un archivo JSON
    with open(path_json, 'w') as json_file:
        json.dump(result, json_file, indent=4)

/home/yeriel/workspace/pretiles/ia/postprocessing/transform.py:69: RankWarning: Polyfit may be poorly conditioned
  coeffs = np.polyfit(x_win, y_win, 1)


In [3]:
# import matplotlib.pyplot as plt
# #mask = dem_info['mask']
# #mask = np.expand_dims(mask, -1)
# o = [o.detach().cpu().numpy()[0,:,:,:].transpose(1, 2, 0) for o in outputs]
        
# o1 = o[0] 
# o1 = (np.clip(o1,0,1)*255).astype(np.uint8)
# o2 = o[1]
# print(o1.shape, o2.shape)

# plt.imshow(o1) 
# plt.show()
# plt.imshow(o2, cmap='gray') 
# plt.show()